<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/Vision/12_01_StableDiffusion_DreamBooth_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!git clone https://github.com/kohya-ss/sd-scripts
%cd sd-scripts

Cloning into 'sd-scripts'...
remote: Enumerating objects: 4954, done.
remote: Counting objects: 100% (2460/2460), done.
remote: Compressing objects: 100% (319/319), done.
remote: Total 4954 (delta 2285), reused 2202 (delta 2140), pack-reused 2494
Receiving objects: 100% (4954/4954), 8.58 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (3499/3499), done.
/content/sd-scripts


In [3]:
!pip install -r requirements.txt
!pip install --upgrade protobuf
!pip install xformers==0.0.22 triton bitsandbytes

Obtaining file:///content/sd-scripts (from -r requirements.txt (line 33))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-clip-torch 2.20.0 requires protobuf<4, but you have protobuf 4.25.1 which is incompatible.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.1 which is incompatible.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION python

env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [5]:
# https://note.com/kohya_ss/n/nb20c5187e15a

In [10]:
dataset = """[general]
enable_bucket = true

[[datasets]]
resolution = 512
batch_size = 4

[[datasets.subsets]]
image_dir = '/content/drive/MyDrive/AI/DB/lora_train_sample_pack/train/20_usu frog'
class_tokens = 'shs frog'
num_repeats = 10

 [[datasets.subsets]]
is_reg = true
image_dir = '/content/drive/MyDrive/AI/DB/lora_train_sample_pack/reg/1_frog'
class_tokens = 'frog'
num_repeats = 1
"""

with open("dataset.toml", "w") as f:
    f.write(dataset)

In [12]:
!accelerate launch --num_cpu_threads_per_process 1 train_db.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1-base"  \
    --dataset_config="dataset.toml" \
    --output_dir="/content/drive/MyDrive/AI/DB/Model/frog" \
    --output_name="frog" \
    --save_model_as=safetensors \
    --prior_loss_weight=1.0 \
    --max_train_steps=1600 \
    --learning_rate=1e-6 \
    --optimizer_type="AdamW8bit" \
    --xformers \
    --mixed_precision="fp16" \
    --cache_latents \
    --gradient_checkpointing \
    --v2

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local